In [1]:
pip install deepxde

In [2]:
# Load libraries
import tensorflow as tf
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt

Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.
Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
# Set figure parameters for all plots
newParams = {'figure.figsize'  : (12, 6),  # Figure size
             'figure.dpi'      : 200,      # figure resolution
             'axes.titlesize'  : 20,       # fontsize of title
             'axes.labelsize'  : 11,       # fontsize of axes labels
             'axes.linewidth'  : 2,        # width of the figure box lines
             'lines.linewidth' : 1,        # width of the plotted lines
             'savefig.dpi'     : 200,      # resolution of a figured saved using plt.savefig(filename)
             'ytick.labelsize' : 11,       # fontsize of tick labels on y axis
             'xtick.labelsize' : 11,       # fontsize of tick labels on x axis
             'legend.fontsize' : 12,       # fontsize of labels in legend
             'legend.frameon'  : True,     # activate frame on lengend?
            }
plt.rcParams.update(newParams) # Set new plotting parameters

In [4]:
# Define Global Parameters
g0    = 9.8                       # gravity
c_d   = .6                        # cd of tennis ball
rho   = 1.                        # density
area  = 3.25*10**-3               # cross sectional area of tennis ball in m^2
mass  = 58 *10**-3                # mass of tennis ball in kg
gamma = .5*(1/mass)*rho*area*c_d  # drag coefficient

# Load data
tdata, xdata, ydata,vxdata,vydata = np.loadtxt("tennis1.txt", unpack=True)

In [5]:
# Define constanst using the tf library
g = tf.constant(g0)
c = tf.constant(0.01)

# Define the ODE system
def ode_system(t, u):
    """
    Define the ODE system:
    du/dt = -g + c*u^2 + 0.1*u
    """
    du_tt = dde.grad.hessian(u, t)
    return du_tt - g + c*u**2 + 0.1*u

# Define the boundary condition
def boundary_l(x, on_boundary):
  return on_boundary and dde.utils.isclose(x[0], 0)

# Define the value of the function at the boundary
def func(t):
  """
  Define the value of the function at the boundary:
  u(0, t) = 0
  """
  return 0

# Define the time domain and declare Dirichlet and Robin BC
geom = dde.geometry.TimeDomain(0, 0.45)
bc_l = dde.icbc.DirichletBC(geom, func, boundary_l)
bc_r = dde.icbc.RobinBC(geom, lambda X, y: 0, boundary_l)

In [6]:
# Load libraries
import tensorflow as tf
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # Import train_test_split


# Load data
tdata, xdata, ydata, vxdata, vydata = np.loadtxt("tennis1.txt", unpack=True)
xdata  = -(xdata-xdata[0])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    tdata, xdata, test_size=0.2, random_state=42
)  # Adjust test_size and random_state as needed

# Create DataSet object with training and testing data
data = dde.data.DataSet(X_train, y_train, X_test, -y_test) # Initialize with train and test data

data = dde.data.PDE(geom,
                    ode_system,
                    [bc_l, bc_r],
                    num_domain=50,
                    num_boundary=2,
                    train_distribution="uniform",
                    solution=None,
                    num_test=500)

layer_size = [1] + [100]*4 + [1]
activation = "sigmoid"
initializer = "Glorot uniform"

NN = dde.maps.FNN(layer_size, activation, initializer)

model = dde.Model(data, NN)
model.compile("adam", lr = 0.0005)

losshistory, train_state = model.train(epochs = 6000)
dde.saveplot(losshistory, train_state, issave = False, isplot = False)

Compiling model...
Building feed-forward neural network...
'build' took 0.302066 s

'compile' took 1.944849 s

Training model...

Step      Train loss                        Test loss                         Test metric
0         [9.73e+01, 4.95e-01, 1.40e-06]    [9.73e+01, 4.95e-01, 1.40e-06]    []  
1000      [2.94e-04, 7.14e-08, 2.44e-07]    [2.30e-04, 7.14e-08, 2.44e-07]    []  
2000      [1.36e-04, 2.29e-09, 5.16e-09]    [1.17e-04, 2.29e-09, 5.16e-09]    []  
3000      [1.28e-04, 1.00e-09, 4.62e-10]    [1.11e-04, 1.00e-09, 4.62e-10]    []  
4000      [1.61e-04, 1.78e-05, 2.25e-08]    [1.39e-04, 1.78e-05, 2.25e-08]    []  
5000      [1.14e-04, 3.62e-10, 2.24e-10]    [9.86e-05, 3.62e-10, 2.24e-10]    []  
6000      [1.06e-04, 2.68e-07, 1.24e-07]    [9.19e-05, 2.68e-07, 1.24e-07]    []  

Best model at step 6000:
  train loss: 1.07e-04
  test loss: 9.23e-05
  test metric: []

'train' took 28.921602 s



In [7]:
# Save prediction of the model
t = geom.uniform_points(100)   # Generate 1000 uniformly spaced points in the time domain
y_pred = model.predict(t)      # Use the trained model to predict values at these points
data_to_save = np.stack((t.flatten(), y_pred.flatten()), axis=1)
np.savetxt('pinn_data.txt', data_to_save, delimiter=',', header='t,y_pred')  # Save the array